# Diffusionsgleichung (2D)

## Einführung

Unter der Annahme, dass die thermischen Parameter konstant sind und wir nur von einer radiogenen Wärmequelle ausgehen, ist die Wärmeleitgleichung in 2-D gegeben durch:

$$
\frac{\partial{T}}{\partial{t}} = \kappa \left( \frac{\partial^2{T}}{\partial{x^2}} + \frac{\partial^2{T}}{\partial{y}^2}\right) + \frac{Q}{\rho c_p}, \tag{1}
$$

wobei $\rho$ die Dichte in [ kg/m<sup>3</sup> ], $c_p$ die spezifische Wärmekapazität in [ J/kg/K ], $\kappa = k/\rho/c_p$ die thermische Diffusivität in [ m<sup>2</sup>/s ] und $Q$ die radiogene Wärmeproduktion pro Volumen in [ W/m<sup>3</sup> ] ist. 

Die Gleichung beschreibt die Änderung der Temperatur mit der Zeit unter der Annahme, dass die Temperatur sich nur durch Diffusion ausbreitet. 

## Das Problem

Nehmen wir and, dass die Gleichung $(1)$ in einem 2D rechteckigen Gebiet mir der Seitenlänge ($L$) 200 km und der Tiefe ($H$) 100 km definiert sei (Abb. 1). Die Temperatur an der Oberfläche sei 0 °C und steigt in dem Gebiet linear mit einem thermischen Gradienten an. Die Randbedingungen der Temperatur an den Seiten sei wählbar zwischen *Dirichlet* und *Neumann* Bedingungen. An der Unterseite des Gebietes nehmen wir in der Mitte des Rechtseckes, auf einer bestimmten Breite ($W_{Plume}$) eine höhere Temperatur als das Umgebungsgestein an. 

<img src="./Figures/Exercise05a_1.png" alt="drawing" width="450"/> <br>
**Abb. 1.** Modelaufbau 

Die beschriebene Konfiguration entspricht in etwa der Situation eines, unterhalb der Lithosphäre, stationär positioniertem Plumekopf. Die überschüssige Temperatur des Plume heizt dabei, mit der Zeit, die Lithosphäre weiter auf. 

Um das Problem mit Hilfe von Julia lösen zu können, müssen wir erst die nötigen Module (```ExtendableSparse, Plots```) und Submodule (```GeoModBox.HeatEquation.TwoD```) definieren: 

In [13]:
using ?

Nun definieren wir die Model ($L$,$H$) und physikalischen Parameter ($k$,$Q$), so wie das Diskretisierungsverfahren (explizit oder implizit):

In [ ]:
FDSchema    =:?
# Physikalischer Parameter ---------------------------------------------- #
P      = (
    L       =   ?,          #   Länge des Models    [m]
    H       =   ?,          #   Höhe des Models     [m]
    k       =   ?,              #   Thermische Konduktivität [W/m/K]
    cp      =   ?,           #   Wärmekapazität [J/kg/K]
    ρ       =   ?,           #   Dichte [kg/m^3]
    K0      =   ?,         #   Kelvin bei 0 C
    Q0      =   ?,              #   Hintergrund Waermeproduktionsrate;
)
P1      = (
    κ       =   ?,   #   Thermische Diffusivität [m^2/s]
    Tbot    =   ?,      #   Temperatur am unteren Rand  [ K ]
    Ttop    =   ?,         #   Temperatur am oberen Rand   [ K ]
    Tplume  =   ?,      #   Temperatur des Plumes [ K ]
    Xanoma  =   ?,            #   x-Koordinate des Zentrums der Anomaly
    Wanoma  =   ?,                #   Breite der Anomaly
    Hanoma  =   ?,               #   Hoehe der Anomaly
    Danoma  =   ?,            #   Tiefe des Zentrums der Anomaly
    Tanoma  =   ?,      #   Temperatur der Anomalie [ C ]
    Qanoma  =   ?              #   Waermeproduktionsrate pro Volumen [W/m^3]
)
P   = merge(P,P1)
# ----------------------------------------------------------------------- #

Als nächstes definieren wir die Anzahl der Gitterpunkte und die Gitterabstände:

In [ ]:
# Numerische Parameter -------------------------------------------------- #
NC      = (
    x       =   ?,             # Gitterpunkte in x-Richtung
    y       =   ?,             # Gitterpunkte in z-Richtung
)
Δ       = (
    x       = ?,
    y       = ?
)
# ----------------------------------------------------------------------- #

Mit deren Hilfe lässt sich das numerische Gitter bestimmen: 

In [ ]:
# Erstellung des Gitters ------------------------------------------------ #
x       = (
    c       =   LinRange(?),
)
y       = (
    c       =   LinRange(?),
)
# ----------------------------------------------------------------------- #

Da wir ein zeit-abhängiges Problem betrachten, müssen wir als nächstes die Parameter für die Zeit definieren. Beachte, für den Fall des expliziten FD-Verfahren, müssen wir das Diffusionsstabilitätskriterium berücksichtigen.

In [ ]:
# Zeit Parameter -------------------------------------------------------- #
T       =   (
    dn      =   25,             #   Inkremente der Graphischen Darstellung,
                                #   d.h. hier nur jeder 25 Zeitschritt
    year    =   365.25*3600*24, #   Sekunden pro Jahr    
    dtfac   =   ?,            #   Multiplikationsfaktor fuer dt
    Δ       =   [0.0],          #   Zeitschrittlaenge
    nt      =   [0]
)
T1      =   (
    tmax    =   60 * 1e6 * T.year,    #   Maximale Laufzeit des Models in s
    Δ       =   T.dtfac * (1.0 / (2.0 * P.κ *(1.0/Δ.x^2 + 1/Δ.y^2)))
)
T           =   merge(T,T1)

nt          =   floor(Int,T.tmax/T.Δ)  # Anzahl der Zeitschritte
time        =   zeros(nt)
# ----------------------------------------------------------------------- #

Zur Visualisierung der Ergebnisse als eine Animation in einer GIF Datei, müssen noch der Ort und der Name der Datei festgelegt werden: 

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("05_2D_Sill_",FDSchema)
save_fig    =   1
# ----------------------------------------------------------------------- #

Als nächstes definieren wir die Anfangsbedingungen und die Datenarrays für unser ProblemÖ

In [19]:
# Anfangstemperaturfeld ------------------------------------------------- #
D       =   (
    Q           =   zeros(?),
    T           =   zeros(?),
    T0          =   zeros(?),
    T_ex        =   zeros(?),
    Tmax        =   zeros(?),
    Tprofile    =   zeros(NC.y,nt)
)
# Hintergrundfeld fuer Waermequellen
D.Q     .=  P.Q0
        
# Temperatur der Lithosphaere - linear zunehmenden mit der Tiefe
for i = 1:NC.x, j = 1:NC.y
    ?
end
D.T0    .=  D.T
# Visualize initial condition ---
p = heatmap(?)

contour!(?)
if save_fig == 0
    display(p)
end
# ----------------------------------------------------------------------- #

## Die Lösung

### Diskretisierung

Um das Problem numerisch lösen zu können, müssen wir unsere Modeldomaine in ein numerisches Gitter unterteilen. Dabei nehmen wir an, das die Temperatur und alle weiteren thermischen Parameter auf den sogenannten *zentralen* Gitterpunkten definiert sind (siehe Abb. 2). Die Aufteilung entspricht der Aufteilung des zeit-unabhängigen Problem.

Zur Lösung unseres Problems benutzen wir wieder die sogenannte *Ghost Nodes* unseres Gitters, welche eine korrekte Einbindung der Randbedingungen ermöglichen. 

#### Gitter und Indizierung

<img src="./Figures/Exercise04_2.png" alt="drawing" width="450"/> <br>
**Abb. 2.** Versetztes Gitternetz. 

Das gegebene versetzte Gitter ermöglicht eine sogenannte *konservative* finite Differenzen Approximation, wobei angenommen wird, dass der Wärmefluss $q_{i,j} = -k \frac{\partial{T}}{\partial{x_{i,j}}}$ auf dem Mittelpunkt der Gitterlinien und die Temperatur im Zentrum einer Gitterzelle definiert ist (streng genommen ist die Wärmeleitfähigkeit dann auch auf den Gitterlinien definiert; da diese allerdings konstant ist, müssen wir das hier nicht berücksichtigen; bei variablen thermischen Parameter muss die Gleichung etwas anders diskreditiert werden). Außerdem können wir durch die zentralen Temperaturgitterpunkte, in Verbindung mit den *Ghost Nodes* relativ einfach Randbedingungen einbinden, die die gleiche Fehlerordnung besitzen, wie die zentralen Differenzenquotienten im Inneren unseres Models. 

Bei der Indizierung unserer Gitterpunkte unterscheiden wir zwischen *lokalen* und *globalen* Indizes. Der lokale Index beschreibt die Position auf dem *i*,*j*-Gitter. Der globale Index, ist ein durchlaufender Index von 1 bis nx*ny und entspricht der Anzahl der Gleichungen, d.h. die Gesamtanzahl der inneren Gitterpunkte. Der globale Index wird auch beim Aufstellen der Koeffizientenmatrix für unser lineares Gleichungssystem verwendet. 

Für jeden Gitterpunkt, also für jede Gleichung, gibt ein sogenannter numerischer Stempel (*stencil*) die Position der Gitterpunkte an, welche für die jeweilige Gleichung gültig sind. Die Koeffizienten für diese Gitterpunkte sind dann jeweils ungleich null und alle anderen gleich null. Die Nomenklatur für den Stempel richtet sich häufig nach der eines Kompases, d.h. wir besitzen Punkte auf: Süden, Westen, Zentral, Osten, und Norden. Für jede Gleichung, ist der globale Index ($ii$) jedes Punktes des Stempels gegeben durch die relative Position zum Zentral Punkt des Stempels, d.h.: 

$$
iS = ii - nx,\\\
iW = ii - 1,\\\   
iZ = ii, \\\
iE = ii + 1,\\\
iN = ii + nx.
$$

#### Finite Differenzen Approximation

Nun können wir die partielle Differentialgleichung durch unsere finiten Differenzen approximieren durch: 

**Explizit (Forward Euler Verfahren)**
$$
T_{i,j}^{n+1} = T_{i,j}^n + \kappa \Delta{t} \left(\frac{T_{i-1,j}^{n} - 2 T_{i,j}^{n} + T_{i+1,j}^{n}}{\Delta{x^2}} \right) + \kappa \Delta{t} \left(\frac{T_{i,j-1}^{n} - 2 T_{i,j}^{n} + T_{i,j+1}^{n}}{\Delta{y^2}} \right) + \frac{Q_{i,j} \Delta{t}}{\rho c_p}, \tag{2}
$$

wobei $n+1$ und $n$ jeweils der neue und er aktuell Zeitschritt ist. 

**Implizit (Backward Euler Verfahren)**
$$
-b T_{i,j-1}^{n+1} - a T_{i-1,j}^{n+1} + (2a + 2b + c) T_{i,j}^{n} - a T_{i+1,j}^{n+1} - b T_{i,j+1}^{n+1} = T_{i,j}^{n} + \frac{Q_{i,j}}{\rho c_p}, \tag{3}
$$

wobei $a = \kappa / \Delta{x^2}$, $b = \kappa / \Delta{y^2}$, und $c = 1 / \Delta{t}$ ist. <br>
D.h. wir haben ein Gleichungssystem mit fünf nicht-null Diagonalen in der Koeffizientenmatrix $\bm{A}$, einem unbekannten Vektor $T^{n+1}$ und einen bekannten Vektor $T^n$ beschrieben werden kann. 

### Randbedingungen

Die Temperatur auf den *Ghost Nodes* ist für *Dirichlet* und *Neumann* Randbedingungen genau so definiert, wie im [stationären 2-D Fall](./04_2D_Diffusion_Stationary.ipynb), bzw. den dazugehörigen [expliziten](./02_1D_Heat_explicit.ipynb) und [impliziten](./03_1D_Heat_implicit.ipynb) 1-D Versionen. <br>
Im expliziten Fall, wird die auf den *Ghost Nodes* berechnete Temperatur in der zu lösende Gleichung für die entsprechenden inneren Gitterpunkte direkt mitverwendet. <br>
Im impliziten Fall, haben wir wieder ein lineares Gleichungssystem und müssen entsprechend die Koeffizienten und die rechte Seite für die Gleichungen der **inneren Gitterpunkte** in der Nähe der Ränder, in Abhängigkeit der Randbedingungen, mit Hilfe der Temperatur auf den *Ghost Nodes* modifizieren (Herleitung siehe Vorlesung).

Bei der Initialisierung der Randbedingungen bediehnung wir uns eines Tricks und definieren die Temperatur auf den *Ghost nodes* später im Skript: 

In [ ]:
# Randbedingungen ------------------------------------------------------- #
BC      =   (type    = (W=:Neumann, E=:Neumann, N=:Dirichlet, S=:Dirichlet),
           val     = (W=zeros(?),E=zeros(?),N=?,S=?))
# ----------------------------------------------------------------------- #

### Lösen des Problems

#### Gleichungssystem

Lasst uns für den impliziten Fall nun die Koeffizientenmatrix und rechte Seite initialisieren: 

In [ ]:
# Linear System of Equations -------------------------------------------- #
Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
ndof    =   ?
K       =   ExtendableSparseMatrix(?,?)
rhs     =   zeros(?)
# ----------------------------------------------------------------------- #

#### Zeitschleife

Numerisch können wir nun in einer Zeitschleife die *PDG* auf unterschiedliche Art und Weise lösen. 

Für den expliziten Fall, wollen wir erst einmal die Gleichungen und Randbedingungen selbst programmieren, bevor wir die eingebauten Funktionen aus dem ```GeoModBox.HeatEquation.TwoD``` Submodul verwenden. 

Für den impliziten Fall, verwenden wir gleich die eingebaute Funktion ```BackwardEuler2Dc!()```. 

In [ ]:
# Zeitschleife ---------------------------------------------------------- #
for n = 1:nt
    println(n)
    # Speicher das Temperaturprofil bei x = L/2
    @. D.Tprofile[:,n]  =   ?
    D.Tmax[n]           =   ?

    if n>1
        if FDSchema==:explicit             
            ?
        elseif FDSchema==:implicit
            BackwardEuler2Dc!(?)
        end                            
        time[n]     =   time[n-1] + T.Δ
    end
    D.T0        .=   D.T
    
    if mod(n,T.dn) == 0 || n == 1 || n == nt
        p = heatmap(?)

        contour!(?)
        if save_fig == 1
            Plots.frame(anim)
        else
            display(p)                        
        end
    end
end

## Results

Zur besseren Darstellung und Analyse der Ergebnisse, stellen wir noch ein vertikales Profil der Temperatur gegenüber der Tiefe, so wie das Maxiumum eines horizontalen Profils der Temperatur gegenüber der Zeit in der jeweiligen Mitte der Domaine graphisch dar: 

In [ ]:
q = plot(?)

plot!(?)

Nun müssen wir noch die Animation erstellen und speichern:

In [ ]:
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)        
    savefig(string("./Results/05_Sill_TProfile_Tmax_",FDSchema,".png"))
else
    display(q)
end
foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))